<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/docs/guidebook/application_development_handbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **_<font color = "red">Agent</font><font color = "blue">ly</font>_ 3.0 Application Development Handbook**
> Don't know what is Agently yet? [>>>  READ THIS FIRST](https://github.com/Maplemx/Agently/blob/main/docs/guidebook/introduction.ipynb)
>
> How to use: `pip install Agently`
>
> Github Repo: https://github.com/Maplemx/Agently
>
> Contact Me: moxin@Agently.cn
>
> If you like this project, please ⭐️ our repo, thanks.

## Quick Start


### Package Installation

> ℹ️ If you're using colab or jupyter, run this package installation first to enable all code down below.

In [ ]:
# Main Package
!pip install Agently
# Requirement Packages
## Network
!pip install aiohttp
!pip install websockets
!pip install tornado
## Model Clients
!pip install openai
!pip install httpx
!pip install erniebot
!pip install zhipuai
## Data Format
!pip install PyYAML

### Hello World

In [ ]:
# Import and Settings
import Agently
agent = Agently.create_agent()
agent\
    .use_model("OpenAI")\
    .set_model("auth", { "api_key": "<Your-API-Key>" })
# Start to use
agent\
    .input("response 'hello world'.")\
    .start()

## Settings

### Where can you set your settings?

Agently framework provide different settings space for developers to use.

In [ ]:
import Agently

# First and most recommended: AgentFactory Settings
## Settings of AgentFactory will be inherit to every agent instance created by
## agent factory instance
agent_factory = Agently.AgentFactory()
## Use key 'current_model' to set model you want to use
## Use key 'model.<model name>.<settings key>' to set single setting
agent_factory\
    .set_settings("current_model", "OpenAI")\
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })

# Second: Agent Settings
## You can give an agent instance unique settings
agent = agent_factory.create_agent()
agent\
    .set_settings("current_model", "ZhipuAI")\
    .set_settings("model.ZhipuAI.auth", { "api_key": "<Your-ZhipuAI-API-Key>" })
## These settings above will overwrite the settings inherit from agent factory
## but will not affect other agent instance created by same agent factory
another_agent = agent_factory.create_agent()
## another_agent will still using the OpenAI settings inherit from agent factory

# Third: Global Settings
## If you have some settings that you want to set for every class(AgentFactory,
## Agent, Request...) in your application, you can use global settings to make
## those settings as default settings
Agently.global_settings\
    .set("current_model", "OpenAI")\
    .set_settings("model.OpenAI.options", { "model": "gpt-3.5-turbo-1106" })
## Now we set 'gpt-3.5-turbo-1106' as default for every OpenAI model request

# The Last One: Request Settings
## Maybe sometimes you just want to use request instance to do some simple
## reques. You can also give request instance unique settings.
request = Agently.Request()
request\
    .set_settings("current_model", "ERNIE")\
    .set_settings("model.ERNIE.auth", {
        "aistudio": "<Your-Baidu-AIStudio-Access-Token>"
    })

### Common Types of Settings

- **Model Settings**:

    Model settings helps developers to configure almost everything they need during model requesting.
    
    **Standard Usage**:
    
    `.set_settings("model.<model name>.<setting key>", <setting value>)`

    **Alias**:
    - `agent.use_model("<model name>")`
    - `agent.set_model("<setting key>", <setting value>)`
    - `agent.set_model_auth({ "<auth key>": "<auth value>" })`
    - `agent.set_model_url("<base url>")`
    - `agent.set_model_option("<option key>", <option value>)`
    - `request.use_model("<model name>")`
    - `request.set_model("<setting key>", <setting value>)`
    - `request.set_model_auth({ "<auth key>": "<auth value>" })`
    - `request.set_model_url("<base url>")`
    - `request.set_model_option("<option key>", <option value>)`

- **Proxy**:
    
    Proxy settings helps developers to use proxy to visit website / request APIs.

    **Standard Usage**:
    
    `.set_settings("proxy", "<proxy address>")`

    **Alias**:
    - `agent_factory.set_proxy("<proxy address>")`
    - `agent.set_proxy("<proxy address>")`
    - `request.set_proxy("<proxy address>")`

- **Component Toggles**:

    Component toggles can be used to turn on / turn off specific agent components. If you turn off an agent component, it will not be loaded and will not paticipate in any agent process stage.

    **Standard Usage**:
    
    `.set_settings("component_toggles.<component name>", <True | False>)`

    **Alias**:
    - `agent_factory.toggle_component("<component name>", <True | False>)`
    - `agent.toggle_component("<component name>", <True | False>)`

- **Plugin Settings**:
    
    Plugin settings can be used to configure specific plugin (not only agent components but also request plugins, storage plugins, etc).

    For example:

    Agent component "Session" need to configure "max length" to decide how long the chat history will be kept in request message.
    
    We can use `.set_settings("plugin_settings.agent_component.Session.max_length", 3000)` to configure it.

    **Standard Usage**:

    `.set_settings("plugin_settings.<plugin type>.<plugin name>.<setting key>", <setting value>)`

- **Debug Mode Toggle**:

    Debug mode toggle can turn on / turn off the debug mode. In debug mode, logs about request data, realtime response from models, JSON parse result and fix request, etc. will be print to the screen.

    > ⚠️: If you turn on debug mode, please remove realtime response printing code like `.on_delta(lambda data: print(data, end=""))` to prevent display conflict.

    **Standard Usage**:

    `.set_settings("is_debug", <True | False>)`

    **Alias**:

    You can turn on debug mode when create agent factory instance by passing paramater `is_debug` like this:

    `agent_factory = Agently.AgentFactory(is_debug=True)`

## Model Request

In this document, we will just use agent_factory settings to demostrate how to make your agent request work with different models. But of course you can choose any other settings methods in your own project if you feel need to.

### OpenAI

#### Chat

In [ ]:
import Agently
agent_factory = Agently.AgentFactory()

# Notice: Remove all annotations before run
agent_factory\
    ## set current model as OpenAI
    ## or you can just remove this setting because "OpenAI" is set by default
    .set_settings("current_model", "OpenAI")\
    ## set your API key
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })\
    ## optional, remove this line if you want to request OpenAI offical API
    ## set value as the base url path you want to change to
    .set_settings("model.OpenAI.url", "https://redirect-service-provider/api/v1")\
    ## optional, set request options followed OpenAI API document's instruction
    .set_settings("model.OpenAI.options", { "model": "gpt-4" })\
    ## optional, important, set this if you want to use proxy!
    ## if you use Clash, VPN, V2Ray to visit OpenAI API, you must check your
    ## client to find your proxy address, then set the address as value here.
    .set_proxy("http://127.0.0.1:7890")

# Test
agent = agent_factory.create_agent()
agent.input("Print 'It works'.").start()

#### Vision

> ⚠️ Notice: If you want to use OpenAI "vision" mode, please make sure your API key has the authority of requesting GPT-4-Vision model.

In [ ]:
import Agently
agent_factory = Agently.AgentFactory()

# Other settings are the same as chat mode above
agent_factory\
    .set_settings("current_model", "OpenAI")\
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })\
    .set_settings("model.OpenAI.options", { "model": "gpt-4-vision-preview" })

# Test
agent = agent_factory.create_agent()
result = agent\
    .files("https://cdn.hk01.com/di/media/images/dw/20200921/384674239925587968.jpeg/KJA2TRK9dzKTpbuXoVyiyz-DjNXw5N9RATMoCwEzKAs?v=w1280")\
    .output({
        "observe": ("String", "Describe what can you see in this picture"),
        "explain": ("String", "Explain how can we thinking about this picture"),
        "tags": [("String", "Classify tag that you will give to this picture")]
    })\
    .start("vision")
for key, content in result.items():
    print(key, ": ", content)

### Microsoft Azure OpenAI

In [ ]:
# Working on it

### Amazon Bedrock Claude

In [ ]:
# Working on it

### ZhipuAI

In [ ]:
import Agently
agent_factory = Agently.AgentFactory()

# Notice: Remove all annotations before run
agent_factory\
    ## set current model as ZhipuAI
    .set_settings("current_model", "ZhipuAI")\
    ## set your API key
    .set_settings("model.ZhipuAI.auth", { "api_key": "<Your-ZhipuAI-API-Key>" })

# Test
agent = agent_factory.create_agent()
agent.input("Print 'It works'.").start()

### Baidu ERNIE

In [ ]:
import Agently
agent_factory = Agently.AgentFactory()

# Notice: Remove all annotations before run
agent_factory\
    ## set current model as ERNIE
    .set_settings("current_model", "ERNIE")\
    ## set your access token
    .set_settings("model.ERNIE.auth", {
        "aistudio": "<Your-AIStudio-Access-Token>",
    })

# Test
agent = agent_factory.create_agent()
agent.input("Print 'It works'.").start()

### MiniMax

In [ ]:
# Not Support Yet

### Xunfei Spark

In [ ]:
# Not Support Yet

## Basic Agent Interact





### Standard Request Slots

In Agently framework, we provide different basic agent interact interfaces in request runtime context to help application developers to express their intention. We named these interfaces "standard request slots" or "slots" for short.

Standard request slots are the bridges between application intention expression and standard model request. Model request plugin developers will put data from these slots into right place in request data / messages as specific model required.

But as application developers, you don't need to worry about that and just need to understand the definition about these slots listed below:

- `prompt_general`: Global instructions that usually need model to follow every time in every request
- `prompt_role`: Descriptions about the role that the model shall play as. For examples: a professional python engineer, a cat girl who loves using emoji, etc.
- `prompt_user_info`: Description about who the user is and what is the user prefer.
- `prompt_abstract`: Abstract and summary about current topic.
- `prompt_chat_history`: Chat logs / history message records of current chat.
- `prompt_input`: Inputs data for model request this time or agent thinking / action this time (short for "this time" in this document).
- `prompt_information`: Information that is useful or you want to add this time.
- `prompt_instruction`: Instructions about what to do / how to do / handle process / rules to follow this time.
- `prompt_output`: Output data structure and explanation for each output item this time.
- `prompt_files`: Path of file(s) you want to quote this time. (Only available when agent or model support file reading)

### Basic Agent Interact Alias

You can update data to standard request slots in request runtime context manually but that is not recommended.

Usually we use interact alias to append data to slots.

**Alias - Slots Mappings**:

These alias can be used by `agent` or `request` instance.

- `.general(any)` => `prompt_general`
- `.role(any)` => `prompt_role`
- `.user_info(any)` => `prompt_user_info`
- `.abstract(any)` => `prompt_abstract`
- `.chat_history(messages: list)` => `prompt_chat_history`
- `.input(any)` => `prompt_input`
- `.info(any)` => `prompt_information`
- `.instruct(any)` => `prompt_instruction`
- `.output(any)` => `prompt_output`
- `.file(file_path: str)` => `prompt_files` (one file path a time)
- `.files(file_path_list: list)` => `prompt_files`(extend file path list)

Basic agent interact alias is the foundation of in-context agent behaviours control. Most agent components are reliant on basic agent interact alias and slots.

### You can pass almost any type of data to agent and receive structure data response

Agently team made a great effort to make sure application developers can pass almost any type of data to agent in those alias.

In [11]:
import Agently
agent_factory = Agently.AgentFactory()
agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })
agent = agent_factory.create_agent()

# You can pass almost any type of data into alias
# list, dict, str, number, bool... whatever you want
role_settings = {
    "name": "Frank",
    "desc": "Frank is always chill and cool. He responses question never using more than 5 words."
}
topic_tag_list = ["daily chatting", "professional skill", "task/job to finish"]
user_input = input("You want to say: ")

# Of course you can pass a variable into the alias
# or construct a dict inside the alias
result = agent\
    .role(role_settings)\
    .input(user_input)\
    .info({ "topic_tag_list": topic_tag_list })\
    .instruct([
        "Response {input} acting follow the {role} settings.",
        "Classify the topic about {input} and {output.response} this time and tag it using the tags in {topic_tag_list}",
    ])\
    .output({
        "response": ("String", "Your direct response as {role}"),
        "tags": [("String in {topic_tag_list}", "Tag by examine {input} and {response}")],
    })\
    .start()
# ("<Type>", "<Description>") is a special expression designed by Agently framework
# to help developers to define the output requirement of a specific item

# Return from agent.start() is a structure data the same as .output() required
# Let's try to print item values of result as it is a dict, to see if it works
print("[Response]:", result["response"])
print("[Topic Tags]:", result["tags"])

You want to say: Hey man, what's up today? Do you wanna go to the supermarket with me today?
[Response]: Not today, maybe later.
[Topic Tags]: ['daily chatting']


## Agent Component

Although basic agent interact provide an easy way to organize request data, when we want to manage agent data in its own life circle or want to enhance agent to make it can complete much more complex task, agent component plugins are your best choice.

In fact, Agently framework provide a runtime environment to ensure many different agent component plugins can be built on it.

Community developers are encouraged to publish plugins and most useful, popular plugins will be updated into this document with the name of the author.

> ℹ️ Notice: Agent component plugins must be used in an agent instance.

### Role



#### Component Information

**Author**: Agently Team

**Plugin File**: [click to view](https://github.com/Maplemx/Agently/blob/main/src/plugins/agent_component/Role.py)

**Desciption**: Agent component "Role" is used to manage who the agent should act as or how the agent should behaviour like.

**Agent Alias**:

- `.set_role_name(name: str)`: set a name for this role settings
- `.set_role(key: str, value: any)`: set a value to specific key in role settings
- `.update_role(key: str, value: any)`: update a value of specific key in role settings
- `.append_role(key: str, value: any)`: append value to a list in role settings
- `.extend_role(key: str, value: list)`: extend list to a list in role settings
- `.save_role(role_name: str=None)`: save this role settings to local storage, if you did not set a name for this role, you can pass a name to `role_name`
- `.load_role(role_name: str)`: load role settings by `role_name` from local storage and put all the settings to current agent

**Participant Stages**:

- `Prefix Stage`: update slot `role`

**Cooperate with Facility**:

[Role Manager](#scrollTo=ivt4xp5_D563)

#### Use Cases

**Use Case 1: Set Role Settings to Change Agent Behaviours**

In [17]:
import Agently
agent_factory = Agently.AgentFactory()
agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })
agent = agent_factory.create_agent()

result = agent\
    .set_role("NEVER RESPONSE ANY WORD EXPECT EMOJIS")\
    .input("Hey, what is your plan today? Give me the details!")\
    .start()
print(result)

🏂🌞🍔🎬😴


**Use Case 2: Save and Load Role Settings**

In [ ]:
# Let's save the role settings in last case as "Emoji Player"
import Agently
agent_factory = Agently.AgentFactory()
agent = agent_factory.create_agent()

agent\
    .set_role("NEVER RESPONSE ANY WORD EXPECT EMOJIS")\
    .save_role("Emoji Player")

In [20]:
# Then we can load the role by name
import Agently
agent_factory = Agently.AgentFactory()
agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })
agent = agent_factory.create_agent()

result = agent\
    .load_role("Emoji Player")\
    .input("How about go fishing right now?")\
    .start()
print(result)

🎣


**Use Case 3: Update Role Settings in Multi Rounds Chat Continually**

In [ ]:
# Since role settings can be updated before .start() every time
# we can make our agent change its acting by update role settings continually
import Agently
agent_factory = Agently.AgentFactory()
agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "<Your-OpenAI-API-Key>" })
agent = agent_factory.create_agent()

emotion = "calm"
while True:
    input_content = input("[YOU]: ")
    if input_content == "#exit":
        print("Bye~👋")
        break
    result = agent\
        .input({
            "input": input_content,
            "emotion": emotion
        })\
        .output({
            "act_according_emotion": ("String", "describe your behaviours will be according {emotion}"),
            "response": ("String", "your response to {input} according {act_according_emotion}"),
            "emotion_change": (
                "String",
                "according user's {input} and your {response},\
                 decide your emotion will remain or change,\
                 then output you current emotion"
            )
        })\
        .start()
    print("[AGENT]: ", result["response"])
    print("[ACT ACCORDING EMOTION]", emotion, ": ", result["act_according_emotion"])
    print("[EMOTION CHANGE] From ", emotion, " to ", result["emotion_change"])
    emotion = result["emotion_change"]

## Facility

Facility is another type of plugins which is usually used to provide a global methods package to help application developers to manage global data in some specific domain and to communicate data with agent components.

> ℹ️ Notice: Facility is independent from agent.

### Role Manager